# Study of a more simple trajectory
We will study a collection of 3 trajectories in order to compare the visualisation of each library

In [32]:
"""
Function to compute the execution time.
"""
from datetime import timedelta
import time
def get_time(f, kwargs={}):
    times = []
    result = None
    for i in range(1):
        print("\nmeasure",i+1)
        t = time.time()
        result = f(**kwargs)
        res = time.time()-t
        times.append(res)
        print( timedelta(seconds=res))
        if res > 3600*2:
            print("trop de temps")
            return result
        
    print("\nall times:", times)
    a = min(times)
    b = max(times)
    print("times' outliers:",a,"&",b)
    t = (sum(times)-a-b)/3
    print("benchmark time",str(t)+"s","or", timedelta(seconds=t))
    return result

In [33]:
import numpy as np
import pandas as pd
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import movingpandas as mpd
#from holoviews import opts, dim
import hvplot
import warnings
warnings.simplefilter("ignore")
from geopy.distance import geodesic 

In [34]:
SIZE = "large"

# cars trajectories

In [135]:
PATH='datasets/walking_journey (copy).csv'
#PATH="datasets/new(1).csv"
df = pd.read_csv(PATH)
df["time"] = pd.to_datetime(df["time"],format="mixed")
traj = mpd.TrajectoryCollection(data=df, traj_id_col='id', t='time', x='longitude', y='latitude')

In [136]:
stop_points = mpd.TrajectoryStopDetector(traj).get_stop_points(min_duration=timedelta(minutes=10), max_diameter=40)
stop_points

,geometry,start_time,end_time,traj_id,duration_s
stop_id,,,,,
0_2023-12-23 14:28:55,POINT (4.36304 50.83640),2023-12-23 14:28:55,2023-12-23 14:39:35,0,640.0
0_2023-12-23 14:47:19,POINT (4.36168 50.83742),2023-12-23 14:47:19,2023-12-23 14:57:27,0,608.0
0_2023-12-23 15:00:55,POINT (4.36255 50.83593),2023-12-23 15:00:55,2023-12-23 15:31:03,0,1808.0


In [137]:
print("nombre de points d'arret",len(stop_points))

nombre de points d'arret 3


In [138]:
traj.trajectories[0].to_point_gdf()

,id,geometry
time,,
2023-12-23 14:14:47,0,POINT (4.36190 50.83815)
2023-12-23 14:15:03,0,POINT (4.36197 50.83809)
2023-12-23 14:15:19,0,POINT (4.36201 50.83806)
2023-12-23 14:15:35,0,POINT (4.36202 50.83803)
2023-12-23 14:15:51,0,POINT (4.36201 50.83801)
...,...,...
2023-12-23 15:43:19,0,POINT (4.36390 50.83512)
2023-12-23 15:43:35,0,POINT (4.36394 50.83512)
2023-12-23 15:43:51,0,POINT (4.36394 50.83514)


# Accuracy

In [139]:
HM = (50.83749695938153, 4.361910678357021)
ICI_PARIS = (50.83639483000942, 4.36310730085289)
PRIMARK = (50.83609678331181, 4.362912067372084)

hm = stop_points["geometry"].values[0]
hm = (hm.y, hm.x)
ici_paris = stop_points["geometry"].values[1]
ici_paris = (ici_paris.y, ici_paris.x)
primark = stop_points["geometry"].values[2]
primark = (primark.y, primark.x)
print(hm, ici_paris, primark)

def dist(coord1, coord2):
    return geodesic(coord1, coord2).m

dist(hm, HM), dist(ici_paris, ICI_PARIS), dist(primark, PRIMARK)

(50.83639700626393, 4.363043524166537) (50.83742489870556, 4.3616806) (50.835932858861504, 4.3625484)


(146.08677385878482, 152.41886300569195, 31.446004198856667)

In [140]:
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import geopandas as gpd

hm_points = [(4.361751392226751,50.837557454216),
(4.361680197932412,50.83758512529476),
(4.361615507694445,50.83749602253124),
(4.361348909126123,50.83758455842273),
(4.361286136778264,50.83746780017795),
(4.361574839843958,50.83725473799701),
(4.36203388271873,50.8374328390111),
(4.361942032549679,50.8375707149803),
(4.361751392226751,50.837557454216)]

primark_points = [(4.362896766180572,50.83617392790824),
(4.362470536721625,50.83596462800733),
(4.362415641019282,50.83600564607535),
(4.361831949209036,50.83568922774191),
(4.36214474170608,50.83547792998715),
(4.362863512141311,50.83587392823398),
(4.3628074875803,50.83591228182476),
(4.363039276765206,50.83603882377207),
(4.362896766180572,50.83617392790824),
]

ici_paris_points = [(4.362977955590129,50.83630268153196),
(4.363328861404733,50.83639980218501),
(4.363276140239741,50.8364930460425),
(4.362918451023474,50.83639022943366),
(4.362977955590129,50.83630268153196)]

hm_polygon = Polygon(hm_points)
primark_polygon = Polygon(primark_points)
ici_paris_polygon = Polygon(ici_paris_points)
d = {'geometry': [hm_polygon,primark_polygon,ici_paris_polygon]}
gdf = GeoDataFrame(d, crs="EPSG:31370")
places = gdf.hvplot(geo=True, color="rgba(245, 40, 145, 0.3)")

In [141]:
traj.trajectories[0].hvplot(title='Trajectory {}'.format(traj.trajectories[0].id), line_width=7.0) * places *stop_points.hvplot(geo=True, size='duration_s', color='deeppink')

:Overlay
   .Tiles.I    :Tiles   [x,y]
   .Path.I     :Path   [Longitude,Latitude]
   .Points.I   :Points   [Longitude,Latitude]   (triangle_angle)
   .Polygons.I :Polygons   [Longitude,Latitude]
   .Points.II  :Points   [Longitude,Latitude]   (duration_s)

In [101]:
gdf.crs

<Projected CRS: EPSG:31370>
Name: BD72 / Belgian Lambert 72
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: Belgium - onshore.
- bounds: (2.5, 49.5, 6.4, 51.51)
Coordinate Operation:
- name: Belgian Lambert 72
- method: Lambert Conic Conformal (2SP)
Datum: Reseau National Belge 1972
- Ellipsoid: International 1924
- Prime Meridian: Greenwich